In [16]:
# Import the libraries we'll use throughout this tutorial
import os
from dotenv import load_dotenv
from strands import Agent
from strands.models import BedrockModel


# Load environment variables (for API keys)
load_dotenv()


True

In [17]:
# bedrock_model = BedrockModel(
#     model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",
#     region_name="us-west-2",
# )

bedrock_model = BedrockModel(
    model_id="us.amazon.nova-micro-v1:0",
    region_name="us-west-2",
)


In [18]:
import textwrap
import os

# Set consistent width
DISPLAY_WIDTH = 125

def print_response(text, title=None, width=DISPLAY_WIDTH):
    """Standard printing function for the notebook"""
    if title:
        print(f"\n🤖 {title}")
        print("─" * width)
    
    wrapped = textwrap.fill(str(text), width=width)
    print(wrapped)
    
    if title:
        print("─" * width)



## Step 1: Your First Agent - The Basic Advisor

Let's start simple. We'll create our first agent that can give basic startup advice without any external tools.



In [25]:
# Create our first basic startup advisor
basic_advisor = Agent(
    system_prompt="""You are a startup advisor with experience in market analysis and business strategy. 
    Help entrepreneurs think through their business ideas by asking good questions and providing practical advice.
    Be encouraging but realistic. Keep your responses concise and to the point - limit your advice to one clear paragraph.""",
    model=bedrock_model,
)

# Simple query
startup_idea = "I want to start a food delivery app for healthy meals"

response = basic_advisor(f"Give me initial thoughts on this startup idea in one paragraph: {startup_idea}");


print("✅ Analysis complete!")
print_response(str(response), "Startup Advisor Response")

Starting a food delivery app for healthy meals is a promising idea given the growing demand for convenient, nutritious food options. To make it successful, consider your target market's specific health needs and preferences, and ensure your menu offers a variety of options to cater to different dietary restrictions and tastes. Partner with local, reputable restaurants or chefs that specialize in healthy cooking to source your meals. Also, focus on a user-friendly app interface, efficient delivery logistics, and robust marketing strategies to attract and retain customers. Remember to comply with health and safety regulations, and continuously gather feedback to improve your offerings and service.🚀 Starting analysis...
✅ Analysis complete!

🤖 Startup Advisor Response
─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Starting a food delivery app for healthy meals is a promising idea given the growing demand for co

## Step 2: Adding Search Tool - The Research Agent

Now we'll upgrade our agent with web search capabilities using Perplexity. This requires setting up an MCP (Model Context Protocol) connection.


In [28]:
# Import additional libraries for MCP connection
from mcp import StdioServerParameters, stdio_client
from strands.tools.mcp import MCPClient


# Check if we have the API key
PERPLEXITY_API_KEY = os.getenv("PERPLEXITY_API_KEY")

if PERPLEXITY_API_KEY:
    print("✅ Perplexity API key found!")
else:
    print("⚠️ Please set your PERPLEXITY_API_KEY in a .env file")


✅ Perplexity API key found!


In [29]:
# Connect to an MCP server using stdio transport
search_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="docker",
            args=[
                "run", "-i", "--rm", 
                "-e", "PERPLEXITY_API_KEY",
                "mcp/perplexity-ask"
            ],
            env={"PERPLEXITY_API_KEY": PERPLEXITY_API_KEY},
        )
    )
)

def print_mcp_tools(tools):
    """
    Display MCP tools in a clean, readable format using the correct attributes
    """
    print("🛠️ Available MCP Tools:")
    print("=" * 50)
    
    for i, tool in enumerate(tools, 1):
        print(f"{i}. {tool.tool_name}")

In [31]:
# Now let's create a research agent that can search the web
with search_client:
    # Get the search tools available
    search_tools = search_client.list_tools_sync()
    print_mcp_tools(search_tools)
    
    # Create a research agent with search capabilities
    research_agent = Agent(
        system_prompt="""You are a market researcher. Use web search to find current information 
        about markets, competitors, and industry trends. Provide factual, up-to-date insights in 100 words.""",
        model=bedrock_model,
        tools=search_tools
    )
    
    print()
    query = "food delivery market size and main competitors"
    print(f"📋 Task: {query}")
    print("=" * 50)
    
    result = research_agent(query)

    print("✅ Analysis complete!")
    print_response(str(result), "Startup Advisor Response")

🛠️ Available MCP Tools:
1. perplexity_ask
2. perplexity_research
3. perplexity_reason
📋 Task: food delivery market size and main competitors
<thinking> To provide insights on the food delivery market size and main competitors, I need to perform a comprehensive research using the provided tools. The Perplexity research tool can help gather current information on market size and competitors. After obtaining the necessary data, I will summarize it to provide a concise answer.</thinking>

Tool #2: perplexity_research
Here's a concise summary based on the latest market research for the food delivery market:

<thinking> I have gathered extensive information from various research tools. Here is a condensed version highlighting the key insights:</thinking>

The global food delivery market size was valued at approximately $288.84 billion in 2024 and is projected to reach $505.50 billion by 2030, with a CAGR of 9.4%. Asia-Pacific, particularly China, dominates the market with $500.50 billion in 

## Step 3: Team Collaboration - The Swarm

Now let's introduce the concept of multiple agents working together. We'll use the "swarm" tool to create a team of specialists.

In [33]:
# Import the swarm tool
from strands_tools import swarm

In [ ]:

with search_client:
    search_tools = search_client.list_tools_sync()
    
    # Create a lead researcher who can coordinate a team
    lead_researcher = Agent(
        system_prompt="""You are a Lead Market Researcher who coordinates research teams.
        Use the swarm tool to delegate different aspects of market research to specialist agents.
        You can create multiple agents to research competitors, market size, trends, etc.""",
        model=bedrock_model,
        tools=search_tools + [swarm]  # Note: we're adding the swarm tool!
    )
    
    # Give it a complex research task
    research_task = """
    Research the food delivery market. I need:
    1. Market size and growth trends
    2. Main competitors and their strategies  
    3. Customer behavior and preferences
    
    
    Use a swarm of 2 agents to divide this work efficiently.
    """
    
    print()
    print(f"📋 Task: {research_task}")
    print("=" * 50)
    
    result = lead_researcher(research_task)
    print("✅ Analysis complete!")
    print_response(str(result), "Startup Advisor Response")



## Step 4: Specialized Roles - The Strategy Team

Now we'll create specialized agents with different roles: a Market Analyst and a Strategic Advisor. Each will have a specific expertise and personality.


In [35]:
def create_market_strategy_workflow(startup_idea):
        
    with search_client:
        search_tools = search_client.list_tools_sync()
        tool_names = [tool.tool_name for tool in search_tools]
        print(f"🛠️ Using tools: {', '.join(tool_names)}")
        
        # Market Analyst - focuses on data and research
        market_analyst = Agent(
            system_prompt="""You are a Market Analyst at a top consulting firm. 
            Your expertise is in gathering and analyzing market data, competitor analysis, 
            and industry trends. You provide detailed, data-driven insights. Use web search 
            to find current market information and support your analysis with specific data points.""",
            model=bedrock_model,
            tools=search_tools + [swarm]
        )

        # Strategic Advisor - focuses on actionable recommendations  
        strategic_advisor = Agent(  # ✅ Note the variable name
            system_prompt="""You are a Strategic Business Advisor with 15 years of experience. 
            You take market research and translate it into clear, actionable business strategies. 
            You focus on practical recommendations for positioning, pricing, marketing, and growth.""",
            model=bedrock_model
        )

        print(f"🚀 Analyzing startup: {startup_idea}")
        print("\n" + "="*60)
        print("STEP 1: Market Analysis")
        print("="*60)

        # Step 1: Market Analyst does research
        analyst_task = f"""
        Conduct concise market research for this startup idea: {startup_idea}
        
        Research and analyze:
        1. Market size and growth trends for the relevant market
        2. Main competitors and their positioning
   
        Use a swarm of 1 agent to gather comprehensive data. Be concise.
        """

        market_research = market_analyst(analyst_task)
        print("🔍 Market Analysis Complete!")

        print("\n" + "="*60)
        print("STEP 2: Strategy Development") 
        print("="*60)

        # Step 2: Strategic Advisor creates strategy
        strategy_task = f"""
        Based on this market research, develop a comprehensive business strategy 
        for the startup idea: {startup_idea}
        
        Market Research Findings:
        {market_research}
        
        Provide:
        1. Positioning strategy
        2. Target customer segments  
        3. Competitive differentiation
        4. Go-to-market recommendations
        5. Key success metrics to track
        """

        business_strategy = strategic_advisor(strategy_task) 
        print("💡 Strategy Development Complete!")
        print(business_strategy)

        return market_research, business_strategy  


startup_idea = "AI-powered meal kit service with personalized recommendations"
research_results, strategy_results = create_market_strategy_workflow(startup_idea)

🛠️ Using tools: perplexity_ask, perplexity_research, perplexity_reason
🚀 Analyzing startup: AI-powered meal kit service with personalized recommendations

STEP 1: Market Analysis
<thinking> To conduct concise market research for the startup idea of an AI-powered meal kit service with personalized recommendations, I will need to gather data on the market size and growth trends, as well as the main competitors and their positioning. I will use the `swarm` tool to coordinate a single agent to gather comprehensive data on these topics.</thinking>

Tool #5: swarm
# Market Analysis: AI-Powered Meal Kit Service

## Market Size and Growth Trends

The meal kit delivery service market demonstrates significant growth potential:

- The global meal kit delivery services market was valued at approximately $15.21 billion in 2021 and is projected to reach $64.4 billion by 2030, growing at a CAGR of 17.4% from 2022 to 2030.
- The North American market alone was valued at $6.22 billion in 2021 and is ex

Here's a concise market research summary for the startup idea of an AI-powered meal kit service with personalized recommendations:

### Market Size and Growth Trends
- **Global Market Size:** The global meal kit delivery services market was valued at approximately $15.21 billion in 2021 and is projected to reach $64.4 billion by 2030, growing at a CAGR of 17.4% from 2022 to 2030.
- **North American Market:** The North American market alone was valued at $6.22 billion in 2021 and is expected to maintain strong growth.
- **Growth Drivers:** Personalization technology in food services is growing at 15-20% annually, with AI-powered recommendations representing a high-growth segment. Consumer interest in personalized nutrition solutions has increased by 65% since 2019.

### Key Competitors and Positioning
1. **HelloFresh**
   - Market leader with 39% market share
   - Recent AI investments to enhance personalization
   - Strong logistics network but limited personalization capabilities

2. 

In [36]:
print_response(str(research_results), "Market Research Response")



🤖 Startup Advisor Response
─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Here's a concise market research summary for the startup idea of an AI-powered meal kit service with personalized
recommendations:  ### Market Size and Growth Trends - **Global Market Size:** The global meal kit delivery services market
was valued at approximately $15.21 billion in 2021 and is projected to reach $64.4 billion by 2030, growing at a CAGR of
17.4% from 2022 to 2030. - **North American Market:** The North American market alone was valued at $6.22 billion in 2021 and
is expected to maintain strong growth. - **Growth Drivers:** Personalization technology in food services is growing at 15-20%
annually, with AI-powered recommendations representing a high-growth segment. Consumer interest in personalized nutrition
solutions has increased by 65% since 2019.  ### Key Competitors and Positioning 1. **HelloFresh**    - Market lead

In [37]:
print_response(str(strategy_results), "Business Strategy Response")


🤖 Business Strategy Response
─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
### Comprehensive Business Strategy for AI-Powered Meal Kit Service with Personalized Recommendations  #### 1. **Positioning
Strategy**  **Brand Positioning:** - **Brand Name:** NutriAI - **Tagline:** "Your Personalized Culinary Companion" -
**Positioning Statement:** NutriAI is the premier AI-powered meal kit service that leverages cutting-edge personalization
technology to deliver tailor-made meal plans that align with your unique dietary preferences, health goals, and lifestyle.
**Unique Selling Proposition (USP):** - **AI-Powered Personalization:** Unlike traditional meal kit services, NutriAI uses
advanced AI algorithms to provide hyper-personalized meal plans that adapt in real-time based on user feedback, dietary
trends, and health goals. - **Health Integration:** Seamless integration with health monitoring devices to offer m

## Step 5: Content Creation and File Operations

Now we'll add agents that can create marketing content and save professional reports to files.

In [41]:
# Import file operation tools
from strands_tools import file_write, editor

# Create content creation team
content_writer = Agent(
    system_prompt="""You are a Creative Marketing Writer with expertise in startup marketing.
    You excel at creating compelling marketing copy, campaign ideas, and promotional content.
    Transform business strategies into engaging narratives that resonate with target audiences.
    Focus on benefits, emotional appeal, and clear calls to action.""",
    model=bedrock_model,
)

report_writer = Agent(
    system_prompt="""You are a Professional Report Writer who creates executive-level documents.
    Take research and strategy outputs and format them into clean, professional markdown reports.
    Use proper structure, headings, bullet points, and formatting for easy reading.
    Save all reports using the file_write tool.""",
    model=bedrock_model,
    tools=[file_write, editor]  # This agent can create and edit files!
)

print("Content writer team:")
print("   ✍️ Content Writer Agent - Marketing copy and campaigns")
print("   📄 Report Writer Agent - Professional documentation")


Content writer team:
   ✍️ Content Writer Agent - Marketing copy and campaigns
   📄 Report Writer Agent - Professional documentation


In [ ]:
# Test content creation and file operations
def test_content_creation():
    """
    Test how we can create marketing content and save reports
    """
    # Use some sample strategy data (in real scenario, this comes from previous steps)
    sample_strategy = """
    Business Strategy for AI-Powered Meal Kit Service:
    
    1. Target Market: Health-conscious millennials and Gen Z (ages 25-40)
    2. Positioning: "Personalized nutrition made simple with AI"
    3. Key Differentiators: AI recipe recommendations, local sourcing, dietary customization
    4. Go-to-Market: Start with urban areas, partner with fitness apps and health influencers
    5. Revenue Model: Subscription tiers from $15-50/week based on meal frequency
    """
    
    print("📝 Creating marketing content...")
    
    # Step 1: Content Writer creates marketing materials
    content_task = f"""
    Based on this business strategy, create compelling marketing content:
    {sample_strategy}
    
    Create:
    1. A catchy tagline and value proposition
    2. Key marketing messages for different channels
    3. A draft for a social media campaign
    4. An elevator pitch for investors
    """
    
    marketing_content = content_writer(content_task)
    print("✍️ Marketing content created!")
    print("\n📋 Marketing Materials:")
    print("-" * 40)
    print(marketing_content[:400] + "..." if len(marketing_content) > 400 else marketing_content)
    
    print("\n📄 Creating professional report...")
    
    # Step 2: Report Writer formats and saves everything
    report_task = f"""
    Create a comprehensive startup advisory report that includes:
    
    Strategy Summary:
    {sample_strategy}
    
    Marketing Materials:
    {marketing_content}
    
    Format this as a professional markdown report with proper headings and structure.
    Save it as 'startup_advisory_report.md' in the current directory.
    """
    
    report_result = report_writer(report_task)
    print("📊 Professional report generated and saved!")
    print(f"💾 Report result: {report_result}")
    
    return marketing_content

# Test the content creation
marketing_materials = test_content_creation()


In [ ]:
def complete_startup_advisory(startup_description):
    """
    Complete end-to-end startup advisory workflow
    """
    print("🚀 COMPLETE STARTUP ADVISORY SYSTEM")
    print("=" * 60)
    print(f"📋 Startup Idea: {startup_description}")
    
    if not search_client or not analyst or not strategist:
        print("⚠️ Full system not available - missing search capabilities")
        return
    
    # PHASE 1: Market Research
    print("\n🔍 PHASE 1: Market Research")
    print("-" * 40)
    
    research_task = f"""
    Conduct comprehensive market research for: {startup_description}
    
    Use a swarm of agents to research:
    1. Market size and growth potential
    2. Competitive landscape and key players
    3. Target customer segments and needs
    4. Industry trends and opportunities
    5. Potential challenges and barriers
    """
    
    market_research = analyst(research_task)
    print("✅ Market research completed")
    
    # PHASE 2: Strategy Development  
    print("\n💡 PHASE 2: Strategy Development")
    print("-" * 40)
    
    strategy_task = f"""
    Develop a comprehensive business strategy based on this research:
    
    Startup: {startup_description}
    Research: {market_research}
    
    Provide detailed recommendations for:
    1. Market positioning and value proposition
    2. Target customer segments and personas
    3. Competitive differentiation strategy
    4. Go-to-market plan and channels
    5. Revenue model and pricing strategy
    6. Key milestones and success metrics
    """
    
    business_strategy = strategist(strategy_task)
    print("✅ Business strategy developed")
    
    # PHASE 3: Marketing Content
    print("\n✍️ PHASE 3: Marketing Content Creation")
    print("-" * 40)
    
    content_task = f"""
    Create compelling marketing materials for: {startup_description}
    
    Based on strategy: {business_strategy}
    
    Develop:
    1. Brand positioning and tagline
    2. Key value propositions for different audiences
    3. Marketing messages for various channels
    4. Social media campaign concepts
    5. Investor pitch summary
    6. Customer testimonial framework
    """
    
    marketing_content = content_writer(content_task)
    print("✅ Marketing content created")
    
    # PHASE 4: Professional Report
    print("\n📄 PHASE 4: Report Generation")
    print("-" * 40)
    
    report_task = f"""
    Create a comprehensive startup advisory report including all our work:
    
    Startup: {startup_description}
    Market Research: {market_research}
    Business Strategy: {business_strategy}
    Marketing Content: {marketing_content}
    
    Format as a professional executive report with:
    - Executive Summary
    - Market Analysis
    - Strategic Recommendations  
    - Marketing Plan
    - Implementation Roadmap
    
    Save as 'complete_startup_advisory_report.md'
    """
    
    final_report = report_writer(report_task)
    print("✅ Professional report generated and saved")
    
    print("\n🎉 STARTUP ADVISORY COMPLETE!")
    print("📋 Deliverables generated:")
    print("   • Market research and analysis")
    print("   • Business strategy and recommendations") 
    print("   • Marketing content and campaigns")
    print("   • Professional advisory report")
    
    return {
        'research': market_research,
        'strategy': business_strategy, 
        'marketing': marketing_content,
        'report': final_report
    }

# Test the complete system
test_startup = "A mobile app that uses AR to help people learn cooking by providing real-time visual guidance and ingredient recognition"

if search_client:
    results = complete_startup_advisory(test_startup)
else:
    print("⚠️ Complete system demo requires Perplexity API key")
    print("🔧 Set your PERPLEXITY_API_KEY to see the full workflow in action!")
